In [6]:
import pandas as pd
import sqlite3
import datetime
import numpy as np
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import PercentFormatter
import math

from urllib.parse import urlparse, parse_qs
mpl.rcParams['figure.dpi'] = 200
okabe_ito = ["#000000", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7"]
sns.set_context('paper')
sns.set_palette(sns.color_palette(okabe_ito))
#https://stackoverflow.com/a/45846841
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])
def make_cdf(series):
    return series.value_counts(normalize=True).sort_index().cumsum()
def make_pdf(series):
    return series.value_counts(normalize=True).sort_index()
from mpl_toolkits.axes_grid1 import make_axes_locatable
from datetime import datetime
mpl.rcParams['figure.dpi'] = 100
import pickle

In [48]:
conn = sqlite3.connect('dataset/web-performance-youtube-merged.db')
measurements = pd.read_sql_query("SELECT * FROM measurements", conn)
failed_lookups = pd.read_sql_query('SELECT * FROM lookups WHERE status <> "NOERROR"', conn)
conn.close()

In [49]:
measurements['timestamp'] = pd.to_datetime(measurements['timestamp'])
measurements['date'] = measurements['timestamp'].dt.date

In [50]:
def count_cache_warming(agg):
    #print(agg)
    found_cw = False
    found_msm = False
    for item in agg:
        if item == 0:
            found_msm = True
        if item == 1:
            found_cw = True
    if found_cw and found_msm:
        return 'cw+msm'
    elif found_cw and not found_msm:
        return 'cw'
    elif not found_cw and not found_msm:
        return 'nothing'
    elif not found_cw and found_msm:
        return 'something went completely wrong'
sanity_check = measurements.groupby(['vantagePoint', 'domain', 'server', 'protocol', 'date'], as_index=False).agg(num_msm_id=('msm_id', pd.Series.nunique), sanity_check=('cacheWarming', lambda x: count_cache_warming(x)))

In [51]:
#measurements.error.str.startswith("failed driver.get() for cache warming ###")
#measurements.error.str.startswith("failed driver.get() ###")
#measurements.error.str.startswith("failed loading player for cache warming ###")
#measurements.error.str.startswith("failed loading player ###")
#measurements.error.str.startswith("failed switching selenium focus to youtube iframe or monitoring loop ###")
measurements['error_'] = "none"
measurements['error_'] = np.where(measurements.error.str.startswith("failed driver.get() for cache warming ###"), "cw: driver.get", measurements['error_'])
measurements['error_'] = np.where(measurements.error.str.startswith("failed driver.get() ###"), "msm: driver.get", measurements['error_'])
measurements['error_'] = np.where(measurements.error.str.startswith("failed loading player for cache warming ###"), "cw: player load", measurements['error_'])
measurements['error_'] = np.where(measurements.error.str.startswith("failed loading player ###"), "msm: player load", measurements['error_'])
measurements['error_'] = np.where(measurements.error.str.startswith("failed switching selenium focus to youtube iframe or monitoring loop ###"), "msm: playback", measurements['error_'])
measurements['error_'].value_counts()

none                237470
cw: player load      15316
msm: playback          861
cw: driver.get          84
msm: player load        59
msm: driver.get         24
Name: error_, dtype: int64

In [52]:
servfail_during_msm = pickle.load( open( "servfail_during_msm.pickle", "rb" ) )
no_start_play_event_during_msm = pickle.load( open( "no_start_play_event_during_msm.pickle", "rb" ) )
no_quality_change_event_during_msm = pickle.load( open( "no_quality_change_event_during_msm.pickle", "rb" ) )
googlevideo_changed_during_msm = pickle.load( open( "googlevideo_changed_during_msm.pickle", "rb" ) )


In [53]:
measurements['error_'] = np.where((measurements.cacheWarming == 0) & (measurements.error_ == 'none') & (measurements.msm_id.isin(servfail_during_msm)), "msm: dns error", measurements['error_'])
measurements['error_'] = np.where((measurements.cacheWarming == 0) & (measurements.error_ == 'none') & (measurements.msm_id.isin(no_start_play_event_during_msm)), "msm: no play event", measurements['error_'])
measurements['error_'] = np.where((measurements.cacheWarming == 0) & (measurements.error_ == 'none') & (measurements.msm_id.isin(no_quality_change_event_during_msm)), "msm: no quality event", measurements['error_'])
measurements['error_'] = np.where((measurements.cacheWarming == 0) & (measurements.error_ == 'none') & (measurements.msm_id.isin(googlevideo_changed_during_msm)), "msm: googlevideo changed during playback", measurements['error_'])

In [54]:
measurements['error_'].value_counts()

none                                        235829
cw: player load                              15316
msm: playback                                  861
msm: googlevideo changed during playback       826
msm: dns error                                 525
msm: no play event                             283
cw: driver.get                                  84
msm: player load                                59
msm: driver.get                                 24
msm: no quality event                            7
Name: error_, dtype: int64

In [56]:
sanity_check_three = measurements.sort_values(['timestamp', 'cacheWarming']).groupby(['vantagePoint', 'domain', 'server', 'protocol', 'date'], as_index=False).agg(msm_ids=('msm_id', pd.Series.unique), errors=('error_', lambda x: ', '.join(list(x.values))))

In [67]:
sanity_check_three['errors'] = np.where(sanity_check_three.errors == 'none', 'none, msm: failed to even run', sanity_check_three['errors'])

In [68]:
sanity_check.sanity_check.value_counts()

cw+msm    119206
cw         15402
Name: sanity_check, dtype: int64

In [69]:
sanity_check_three.errors.value_counts()

none, none                                        116621
cw: player load                                    15316
none, msm: playback                                  861
none, msm: googlevideo changed during playback       826
none, msm: dns error                                 525
none, msm: no play event                             283
cw: driver.get                                        84
none, msm: player load                                59
none, msm: driver.get                                 24
none, msm: no quality event                            7
none, msm: failed to even run                          2
Name: errors, dtype: int64

In [63]:
sanity_check_three.groupby('protocol').size()

protocol
https    26921
quic     26922
tcp      26922
tls      26922
udp      26921
dtype: int64

In [64]:
sanity_check_three[sanity_check_three.errors != 'none, none'].groupby('protocol').size()

protocol
https    3469
quic     3413
tcp      3631
tls      3405
udp      4069
dtype: int64

In [71]:
sanity_check_three[sanity_check_three.errors.isin(['none, none', 'none, msm: googlevideo changed during playback', 'none, msm: dns error', 'none, msm: no play event', 'none, msm: no quality event', 'none, msm: failed to even run'])].errors.value_counts()

none, none                                        116621
none, msm: googlevideo changed during playback       826
none, msm: dns error                                 525
none, msm: no play event                             283
none, msm: no quality event                            7
none, msm: failed to even run                          2
Name: errors, dtype: int64

In [78]:
sanity_check_three[sanity_check_three.errors.isin(['none, none', 'none, msm: googlevideo changed during playback', 'none, msm: dns error', 'none, msm: no play event', 'none, msm: no quality event', 'none, msm: failed to even run'])].groupby('protocol').size()

protocol
https    23770
quic     23801
tcp      23468
tls      23843
udp      23382
dtype: int64

In [79]:
sanity_check_three[sanity_check_three.errors.isin(['none, msm: googlevideo changed during playback', 'none, msm: dns error', 'none, msm: no play event', 'none, msm: no quality event', 'none, msm: failed to even run'])].groupby('protocol').size()

protocol
https    318
quic     292
tcp      177
tls      326
udp      530
dtype: int64

In [77]:
sanity_check_three[sanity_check_three.errors == 'none, none'].groupby('protocol').size()

protocol
https    23452
quic     23509
tcp      23291
tls      23517
udp      22852
dtype: int64

In [66]:
sanity_check_three.groupby('protocol').errors.value_counts()

protocol  errors                                        
https     none, none                                        23452
          cw: player load                                    2963
          none, msm: googlevideo changed during playback      191
          none, msm: playback                                 176
          none, msm: no play event                             76
          none, msm: dns error                                 48
          none, msm: player load                                7
          none, msm: driver.get                                 3
          cw: driver.get                                        2
          none, msm: no quality event                           2
          none                                                  1
quic      none, none                                        23509
          cw: player load                                    2944
          none, msm: googlevideo changed during playback      170
          none, msm

In [59]:
sanity_check_three[sanity_check_three.errors == 'none']

,vantagePoint,domain,server,protocol,date,msm_ids,errors
72415,Europe Central,lqiN98z6Dak,188.68.59.46,tls,2022-04-23,054af061-eada-2266-e54c-7cba3fa35c36,none
124923,US West,aqz-KE-bpKQ,94.140.15.16,https,2022-04-23,fe6b59ad-78fa-0549-a0cf-516f3d59ef95,none


In [60]:
measurements[measurements.msm_id == '054af061-eada-2266-e54c-7cba3fa35c36']

,msm_id,py_time,js_time,resource_time_origin,protocol,server,domain,vantagePoint,timestamp,suggested_quality,player_width,player_height,start_time,play_time,video_ids,cacheWarming,error,date,error_
138039,054af061-eada-2266-e54c-7cba3fa35c36,-1,-1.0,-1.0,tls,188.68.59.46,lqiN98z6Dak,Europe Central,2022-04-23 10:17:55.597571,auto,1280,720,0,5,lqiN98z6Dak,1,,2022-04-23,none


In [ ]:
def cw_msm_errors(grouped):
    #print(grouped)
    ret = {}
    ret['sanity_check'] = ""
    ret['cw_id'] = ""
    ret['msm_id'] = ""
    
    cw_error = False
    for cache_warming in [1,0]:
        if cache_warming == 1:
            ret['cw_id'] = grouped[grouped.cacheWarming == cache_warming].msm_id.values[0]
            if grouped[grouped.cacheWarming == cache_warming].error_.values[0] != 'none':
                cw_error=True
        else:
            if cw_error:
                if len(grouped[grouped.cacheWarming == cache_warming]) > 0:
                    ret['msm_id'] = grouped[grouped.cacheWarming == cache_warming].msm_id.values[0]
                    ret['sanity_check'] = "cache warming error but also ran measurement: "+grouped[grouped.cacheWarming == 1].error_.values[0] + " -- "+grouped[grouped.cacheWarming == 0].error_.values[0]
                else:
                    ret['sanity_check'] = "cache warming error: "+grouped[grouped.cacheWarming == 1].error_.values[0]
            else:
                if len(grouped[grouped.cacheWarming == cache_warming]) == 0:
                    ret['sanity_check'] = "cache warming successful but no measurement"
                else:
                    ret['msm_id'] = grouped[grouped.cacheWarming == cache_warming].msm_id.values[0]
                    ret['sanity_check'] = "measurement error: "+grouped[grouped.cacheWarming == cache_warming].error_.values[0]
                
    return pd.Series(ret, index=['sanity_check', 'cw_id', 'msm_id'])
sanity_check_two = measurements.groupby(['vantagePoint', 'domain', 'server', 'protocol', 'date'], as_index=False).apply(cw_msm_errors)
sanity_check_two

In [9]:
measurements[measurements.error_ == 'none'].cacheWarming.value_counts()

1    119079
0    116621
Name: cacheWarming, dtype: int64

In [55]:
measurements[measurements.cacheWarming == 1].groupby('protocol', as_index=False).agg(successful_msms=('msm_id', pd.Series.nunique)).sort_values('protocol')

,protocol,successful_msms
0,https,26921
1,quic,26922
2,tcp,26922
3,tls,26922
4,udp,26921


In [17]:
measurements[(measurements.error_ != 'none')].groupby('protocol', as_index=False).agg(successful_msms=('msm_id', pd.Series.nunique)).sort_values('protocol')

,protocol,successful_msms
0,https,3484
1,quic,3432
2,tcp,3654
3,tls,3420
4,udp,4124


In [22]:
measurements[(measurements.cacheWarming == 0)].groupby('protocol', as_index=False).agg(successful_msms=('msm_id', pd.Series.nunique)).sort_values('protocol')

,protocol,successful_msms
0,https,23955
1,quic,23975
2,tcp,23650
3,tls,24051
4,udp,23575


In [18]:
measurements[(measurements.error_ == 'none') & (measurements.cacheWarming == 0)].groupby('protocol', as_index=False).agg(successful_msms=('msm_id', pd.Series.nunique)).sort_values('protocol')

,protocol,successful_msms
0,https,23452
1,quic,23509
2,tcp,23291
3,tls,23517
4,udp,22852


In [19]:
measurements[(measurements.error_ != 'none') & (measurements.cacheWarming == 0)].groupby('protocol', as_index=False).agg(successful_msms=('msm_id', pd.Series.nunique)).sort_values('protocol')

,protocol,successful_msms
0,https,503
1,quic,466
2,tcp,359
3,tls,534
4,udp,723


In [20]:
measurements[(measurements.error_ == 'none') & (measurements.cacheWarming == 1)].groupby('protocol', as_index=False).agg(successful_msms=('msm_id', pd.Series.nunique)).sort_values('protocol')

,protocol,successful_msms
0,https,23940
1,quic,23956
2,tcp,23627
3,tls,24036
4,udp,23520


In [21]:
measurements[(measurements.error_ != 'none') & (measurements.cacheWarming == 1)].groupby('protocol', as_index=False).agg(successful_msms=('msm_id', pd.Series.nunique)).sort_values('protocol')

,protocol,successful_msms
0,https,2981
1,quic,2966
2,tcp,3295
3,tls,2886
4,udp,3401


In [4]:
measurements[measurements.error_ != 'none'].groupby('protocol', as_index=False).agg(failed_msms=('msm_id', pd.Series.nunique))

,protocol,failed_msms
0,https,3151
1,quic,3121
2,tcp,3454
3,tls,3079
4,udp,3539


In [5]:
measurements[measurements.error_ == 'none'].groupby('protocol', as_index=False).agg(successful_msms=('msm_id', pd.Series.nunique)).sort_values('successful_msms')

,protocol,successful_msms
4,udp,46957
2,tcp,47118
0,https,47725
1,quic,47776
3,tls,47894


In [7]:
measurements[measurements.error_ != 'none'].groupby('vantagePoint', as_index=False).agg(failed_msms=('msm_id', pd.Series.nunique))

,vantagePoint,failed_msms
0,Africa South,2870
1,Asia Pacific Northeast,2285
2,Asia Pacific Southeast,2336
3,Europe Central,2261
4,South America East,2102
5,US East,2317
6,US West,2173


In [36]:
measurements[measurements.error_.str.startswith('cw')].groupby('vantagePoint', as_index=False).agg(failed_msms=('msm_id', pd.Series.nunique))

,vantagePoint,failed_msms
0,Africa South,2347
1,Asia Pacific Northeast,2138
2,Asia Pacific Southeast,2148
3,Europe Central,2235
4,South America East,2077
5,US East,2299
6,US West,2156


In [35]:
measurements[measurements.error_.str.startswith('msm')].groupby('vantagePoint', as_index=False).agg(failed_msms=('msm_id', pd.Series.nunique))

,vantagePoint,failed_msms
0,Africa South,523
1,Asia Pacific Northeast,147
2,Asia Pacific Southeast,188
3,Europe Central,26
4,South America East,25
5,US East,18
6,US West,17


In [29]:
measurements[measurements.error_ == 'none'].groupby('vantagePoint', as_index=False).agg(successful_msms=('msm_id', pd.Series.nunique)).sort_values('successful_msms')

,vantagePoint,successful_msms
0,Africa South,33243
5,US East,33744
1,Asia Pacific Northeast,33977
3,Europe Central,34041
2,Asia Pacific Southeast,34054
6,US West,34150
4,South America East,34261


In [8]:
server_failure_vp_count = measurements[measurements.error_ != 'none'].groupby(['server'], as_index=False).agg(unique_vps=('vantagePoint', pd.Series.nunique))
server_failure_vp_count


,server,unique_vps
0,101.32.27.77,7
1,103.105.98.141,2
2,103.123.108.197,7
3,103.124.95.99,2
4,103.172.17.225,3
...,...,...
121,83.212.102.207,1
122,89.58.10.169,7
123,91.239.27.199,1
124,93.115.24.205,7


In [9]:
server_failure_vp_count.unique_vps.value_counts()

1    59
7    35
2    14
3     7
6     6
4     3
5     2
Name: unique_vps, dtype: int64

In [10]:
servers_that_fail_on_single_vm = server_failure_vp_count[server_failure_vp_count.unique_vps == 1].server.unique().tolist()

In [11]:
measurements[(measurements.error_ != 'none') & (measurements.server.isin(servers_that_fail_on_single_vm))].vantagePoint.value_counts()

Africa South              237
Europe Central            145
Asia Pacific Southeast     75
South America East         15
Asia Pacific Northeast      9
US West                     8
US East                     6
Name: vantagePoint, dtype: int64

In [12]:
measurements[(measurements.error_ != 'none') & (measurements.server.isin(servers_that_fail_on_single_vm)) & (measurements.vantagePoint == "Africa South")].error_.value_counts()

cw: player load    226
msm: playback        7
cw: driver.get       3
msm: driver.get      1
Name: error_, dtype: int64

In [13]:
measurements[(measurements.error_ != 'none') & (measurements.server.isin(servers_that_fail_on_single_vm)) & (measurements.vantagePoint == "Europe Central")].error_.value_counts()

cw: player load    140
msm: playback        5
Name: error_, dtype: int64

In [14]:
measurements[(measurements.error_ != 'none') & (measurements.server.isin(servers_that_fail_on_single_vm)) & (measurements.vantagePoint == "US East")].error_.value_counts()

msm: playback      3
cw: player load    3
Name: error_, dtype: int64

In [15]:
measurements[(measurements.error_ != 'none') & (measurements.server.isin(servers_that_fail_on_single_vm)) & (measurements.vantagePoint == "Asia Pacific Northeast")].error_.value_counts()

msm: playback      5
cw: player load    4
Name: error_, dtype: int64

In [16]:
servers_that_fail_on_every_vm = server_failure_vp_count[server_failure_vp_count.unique_vps == 7].server.unique().tolist()

In [17]:
measurements[(measurements.error_ != 'none') & (measurements.server.isin(servers_that_fail_on_every_vm))].vantagePoint.value_counts()

Europe Central            1895
Africa South              1879
Asia Pacific Southeast    1874
US West                   1868
South America East        1847
US East                   1841
Asia Pacific Northeast    1798
Name: vantagePoint, dtype: int64

In [18]:
measurements[(measurements.error_ != 'none') & (measurements.server.isin(servers_that_fail_on_every_vm))].error_.value_counts()

cw: player load     12895
msm: player load       50
cw: driver.get         31
msm: playback          15
msm: driver.get        11
Name: error_, dtype: int64

In [19]:
measurements[measurements.error == ''].server.nunique()

260

In [20]:
measurements[measurements.error != ''].server.nunique()

126

In [21]:
measurements[measurements.server.isin(measurements[measurements.error == ''].server.unique())].protocol.value_counts()

tls      49163
quic     49087
https    49066
tcp      48762
udp      48686
Name: protocol, dtype: int64

In [22]:
measurements.dtypes

msm_id                   object
py_time                   int64
js_time                 float64
resource_time_origin    float64
protocol                 object
server                   object
domain                   object
vantagePoint             object
timestamp                object
suggested_quality        object
player_width              int64
player_height             int64
start_time                int64
play_time                 int64
video_ids                object
cacheWarming              int64
error                    object
error_                   object
dtype: object

In [23]:
measurements[measurements.error == ''].groupby(['server', 'protocol', 'vantagePoint'], as_index=False).agg(num_msm=('msm_id',pd.Series.nunique)).sort_values('num_msm')

,server,protocol,vantagePoint,num_msm
4347,185.254.18.242,tls,Africa South,1
6888,47.107.121.125,udp,Asia Pacific Southeast,1
4993,199.101.171.125,https,South America East,2
6861,47.103.26.225,tls,Asia Pacific Southeast,2
863,119.3.92.152,tls,Asia Pacific Southeast,2
...,...,...,...,...
3096,172.104.183.19,udp,Asia Pacific Northeast,28
3095,172.104.183.19,udp,Africa South,28
3094,172.104.183.19,tls,US West,28
3092,172.104.183.19,tls,South America East,28


In [24]:
successful_msms = measurements[(measurements.error == '') & (measurements.cacheWarming == 0)].groupby(['server', 'protocol', 'vantagePoint'], as_index=False).agg(num_msm=('msm_id',pd.Series.nunique))

In [30]:
successful_msms[successful_msms.num_msm < 14].server.nunique()

115

In [31]:
resolvers_with_full_results = dict()

for vp in successful_msms.vantagePoint.unique():
    resolvers_with_full_results[vp] = set()
    
    for server in successful_msms.server.unique():
        found_incomplete=False
        for protocol in successful_msms.protocol.unique():
            df_tmp = successful_msms[(successful_msms.server == server) & (successful_msms.vantagePoint == vp) & (successful_msms.protocol == protocol)]
            if len(df_tmp) == 0:
                found_incomplete = True
                break
            else:
                num_msm = df_tmp.num_msm.values[0]
                if num_msm < 14:
                    found_incomplete = True
                    break
        if not found_incomplete:
            resolvers_with_full_results[vp].add(server)

In [32]:
resolvers_with_full_results_set = set()
for vp in successful_msms.vantagePoint.unique():
    if len(resolvers_with_full_results_set) == 0:
        resolvers_with_full_results_set = resolvers_with_full_results[vp]
    else:
        resolvers_with_full_results_set = resolvers_with_full_results_set.intersection(resolvers_with_full_results[vp])
resolvers_with_full_results_set

{'103.232.207.2',
 '104.238.154.123',
 '104.244.72.39',
 '107.172.30.115',
 '107.191.51.151',
 '108.166.219.158',
 '109.205.178.178',
 '128.199.115.141',
 '13.250.108.212',
 '130.61.24.160',
 '134.209.81.226',
 '135.125.181.234',
 '135.125.236.23',
 '138.2.72.170',
 '138.2.79.62',
 '138.2.91.167',
 '139.162.70.69',
 '140.238.10.79',
 '140.238.42.214',
 '141.95.140.195',
 '146.56.112.4',
 '146.56.176.39',
 '146.56.177.76',
 '146.56.184.134',
 '146.59.226.118',
 '146.59.23.151',
 '146.59.32.198',
 '147.189.137.154',
 '150.230.103.62',
 '150.230.197.114',
 '150.230.99.64',
 '152.70.189.89',
 '152.70.232.216',
 '154.49.211.118',
 '16.162.25.97',
 '160.16.123.42',
 '164.90.199.170',
 '168.138.36.90',
 '168.138.50.5',
 '168.138.53.111',
 '172.105.101.43',
 '173.255.211.87',
 '173.82.151.158',
 '173.82.243.163',
 '173.82.68.70',
 '176.103.130.130',
 '176.103.130.131',
 '176.103.130.132',
 '176.103.130.136',
 '176.103.130.149',
 '176.103.130.150',
 '176.103.134.134',
 '176.103.134.135',
 '176.

In [33]:
len(resolvers_with_full_results_set)

134

In [34]:
resolvers_with_half_results = dict()

for vp in successful_msms.vantagePoint.unique():
    resolvers_with_half_results[vp] = set()
    
    for server in successful_msms.server.unique():
        found_incomplete=False
        for protocol in successful_msms.protocol.unique():
            df_tmp = successful_msms[(successful_msms.server == server) & (successful_msms.vantagePoint == vp) & (successful_msms.protocol == protocol)]
            if len(df_tmp) == 0:
                found_incomplete = True
                break
            else:
                num_msm = df_tmp.num_msm.values[0]
                if num_msm < 7:
                    found_incomplete = True
                    break
        if not found_incomplete:
            resolvers_with_half_results[vp].add(server)
resolvers_with_half_results_set = set()
for vp in successful_msms.vantagePoint.unique():
    if len(resolvers_with_half_results_set) == 0:
        resolvers_with_half_results_set = resolvers_with_half_results[vp]
    else:
        resolvers_with_half_results_set = resolvers_with_half_results_set.intersection(resolvers_with_half_results[vp])
print(resolvers_with_half_results_set)
print(len(resolvers_with_half_results_set))

{'139.162.70.69', '94.140.14.49', '176.103.130.131', '185.17.3.188', '45.55.104.59', '43.129.73.66', '210.247.245.78', '37.235.49.163', '176.103.130.130', '103.232.207.3', '45.79.151.58', '150.230.103.62', '176.113.83.49', '103.78.123.211', '104.244.79.105', '147.189.137.154', '146.56.112.4', '194.67.121.139', '103.172.17.226', '159.223.136.223', '83.212.102.207', '80.85.136.144', '67.205.154.232', '116.118.44.237', '176.103.134.5', '149.56.45.165', '138.3.219.11', '37.1.195.86', '150.230.99.64', '160.16.123.42', '192.26.105.29', '94.140.14.141', '64.112.126.239', '35.156.129.204', '188.166.209.249', '103.123.108.197', '43.154.154.162', '103.176.79.211', '185.121.25.64', '185.175.57.120', '217.160.68.68', '176.103.130.132', '94.140.14.140', '138.2.84.109', '193.201.126.42', '188.68.45.12', '45.129.2.32', '80.209.236.92', '85.235.65.70', '104.238.154.123', '103.172.17.225', '31.44.0.4', '185.177.218.107', '13.250.108.212', '47.254.64.251', '180.93.137.1', '94.140.15.16', '199.115.228.98